# Preference Update Bandit

In [1]:
# Import modules.
from abc import ABC, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import import_ipynb
from abstract_bandit import Bandit

importing Jupyter notebook from abstract_bandit.ipynb


In [ ]:
class PrefBandit(Bandit):